In [5]:
import pandas as pd
from datetime import datetime, timedelta
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Load the CSV file
df = pd.read_csv('PRG Sales Comparables Table.csv', encoding='latin1')

# Define new column names
new_column_names = [
    "PRG", "County", 'Use', "Address", "City", "State", "Tax ID1", "Tax ID2", 
    "Map/Grid/Parcel", "Legal Desc.", "Grantor", "Grantee", "Sale Date", 
    "Deed", "Property Rights", "Conditions at Sale", "Time on Market", 
    "Sale Price", "Price/GBA", "Vacancy", "NOI", "Cap Rate", "Land Size (SF)", 
    "GBA (SF)", "Year Built", "Parking", "Zone", "Utilities", "Apt", 
    "Apt2", "Apt Only", "Apt Only2", "Comments"
]

# Rename the columns
df.columns = new_column_names

# Prompt the user to select a value
selection = input("Enter PRG Reference Number: ")

# Check if the selection is valid
if selection in df['PRG'].values:
    # Create a new DataFrame with the row(s) corresponding to the user's selection
    new_df = df[df['PRG'] == selection]

    # Check if new_df is not empty
    if not new_df.empty:
        # Extract values for each column
        PRG = new_df.iloc[0]['PRG']
        County = new_df.iloc[0]['County']
        Use = new_df.iloc[0]['Use']
        Address = new_df.iloc[0]['Address']
        City = new_df.iloc[0]['City']
        State = new_df.iloc[0]['State']
        Tax_ID1 = f"{int(new_df.iloc[0]['Tax ID1']):02}"
        Tax_ID2 = f"{int(new_df.iloc[0]['Tax ID2']):06}"
        Map_Grid_Parcel = new_df.iloc[0]['Map/Grid/Parcel']
        Legal_Desc = new_df.iloc[0]['Legal Desc.']
        Grantor = new_df.iloc[0]['Grantor']
        Grantee = new_df.iloc[0]['Grantee']
        excel_serial_date = new_df.iloc[0]['Sale Date']
        if isinstance(excel_serial_date, str):
            excel_serial_date = int(excel_serial_date)
        start_date = datetime(1900, 1, 1)
        adjusted_date = start_date + timedelta(days=excel_serial_date - 2)
        Sale_Date = adjusted_date.strftime("%m/%d/%Y")
        Deed = new_df.iloc[0]['Deed']
        Property_Rights = new_df.iloc[0]['Property Rights']
        Conditions_at_Sale = new_df.iloc[0]['Conditions at Sale']
        Time_on_Market = new_df.iloc[0]['Time on Market']
        Sale_Price = new_df.iloc[0]['Sale Price']
        Sale_Price = f"${Sale_Price:,.0f}"
        Price_GBA = new_df.iloc[0]['Price/GBA']
        Price_GBA = f"${Price_GBA:,.2f}"
        Vacancy = new_df.iloc[0]['Vacancy']
        NOI = new_df.iloc[0]['NOI']
        Cap_Rate = new_df.iloc[0]['Cap Rate']
        Land_Size_SF = new_df.iloc[0]['Land Size (SF)']
        GBA_SF = new_df.iloc[0]['GBA (SF)']
        Year_Built = new_df.iloc[0]['Year Built']
        Parking = new_df.iloc[0]['Parking']
        Zone = new_df.iloc[0]['Zone']
        Utilities = new_df.iloc[0]['Utilities']
        Apt = new_df.iloc[0]['Apt']
        Apt2 = new_df.iloc[0]['Apt2']
        Apt_Only = new_df.iloc[0]['Apt Only']
        Apt_Only2 = new_df.iloc[0]['Apt Only2']
        Comments = new_df.iloc[0]['Comments']

# Helper Functions

def set_cell_borders(cell, border_color="000000", border_width=4):
    """Set the borders for a cell."""
    tc_pr = cell._element.xpath('.//w:tcPr')
    if not tc_pr:
        tc_pr = OxmlElement('w:tcPr')
        cell._element.insert(0, tc_pr)
    else:
        tc_pr = tc_pr[0]
    
    borders = OxmlElement('w:tcBorders')
    border_attrs = {
        'w:top': border_color,
        'w:left': border_color,
        'w:bottom': border_color,
        'w:right': border_color,
        'w:insideH': border_color,
        'w:insideV': border_color
    }
    
    for border in border_attrs:
        border_elem = OxmlElement(border)
        border_elem.set(qn('w:val'), 'single')
        border_elem.set(qn('w:sz'), str(border_width))  # Border width (1/8 pt)
        border_elem.set(qn('w:space'), '0')
        borders.append(border_elem)

    tc_pr.append(borders)

def set_cell_fill(cell, color):
    """Set the fill color for a cell."""
    shading = OxmlElement('w:shd')
    shading.set(qn('w:fill'), color)
    cell._element.get_or_add_tcPr().append(shading)

def set_paragraph_spacing(cell, spacing_before=0, spacing_after=0):
    """Set paragraph spacing within a cell."""
    for para in cell.paragraphs:
        para_format = para.paragraph_format
        para_format.space_after = Pt(spacing_after)
        para_format.space_before = Pt(spacing_before)
        para_format.line_spacing = Pt(12)  # Single line spacing (12 pt)

def clear_cell_text(cell):
    """Clear text from a cell and set paragraph spacing."""
    cell.text = ''
    set_paragraph_spacing(cell)  # Ensure paragraph spacing is set

def set_cell_text(cell, text, font_name='Times New Roman', font_size=11, bold=True, alignment='left'):
    """Set text and formatting in a cell."""
    cell.text = text
    
    if alignment == 'left':
        align = WD_ALIGN_PARAGRAPH.LEFT
    elif alignment == 'center':
        align = WD_ALIGN_PARAGRAPH.CENTER
    elif alignment == 'right':
        align = WD_ALIGN_PARAGRAPH.RIGHT
    else:
        raise ValueError("Invalid alignment value. Choose from 'left', 'center', or 'right'.")

    for para in cell.paragraphs:
        para_format = para.paragraph_format
        para_format.alignment = align
        para_format.space_after = Pt(0)
        
        for run in para.runs:
            run.font.name = font_name
            run.font.size = Pt(font_size)
            run.font.bold = bold
        
        para.style.font.name = font_name

# Table Manipulation Functions

def fill_row_with_cells(table, row_idx, fill_color="FFFFFF"):
    """Fill a row with cells, setting borders, fill color, and clearing text."""
    for col_idx in range(2):  # Two columns
        cell = table.cell(row_idx, col_idx)
        set_cell_borders(cell, border_color="000000", border_width=4)
        set_cell_fill(cell, color=fill_color)
        clear_cell_text(cell)

def handle_alternating_rows(table, start_row, end_row, color):
    """Handle alternating row colors."""
    for row_idx in range(start_row, end_row + 1):
        fill_row_with_cells(table, row_idx, color)

def add_merged_row(table, row_idx, fill_color="D0D0D0"):
    """Add a merged row with a specific fill color."""
    cell_1 = table.cell(row_idx, 0)
    cell_2 = table.cell(row_idx, 1)
    cell_1.merge(cell_2)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color=fill_color)
    clear_cell_text(cell_1)

# Document Creation Function

def create_table_with_content(doc):
    """Create a table with specified content and formatting."""
    # Set document margins
    section = doc.sections[0]
    section.left_margin = Inches(1)
    section.right_margin = Inches(1)

    # Add a table with 35 rows and 2 columns
    table = doc.add_table(rows=35, cols=2)

    # Row 0
    cell_1 = table.cell(0, 0)
    cell_2 = table.cell(0, 1)
    cell_1.merge(cell_2)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="D0D0D0")
    set_cell_text(cell_1, f"SALE COMPARABLE\nPRG Reference: {PRG}", font_name='Times New Roman', font_size=11, bold=True, alignment='left')

    # Adjust paragraph spacing after PRG Reference
    for para in cell_1.paragraphs:
        if "PRG Reference" in para.text:
            para_format = para.paragraph_format
            para_format.space_after = Pt(6)  # Add 6pt space after PRG Reference line

    # Row 1
    cell_1 = table.cell(1, 0)
    cell_2 = table.cell(1, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "", font_name='Times New Roman', font_size=10, bold=False, alignment='center')
    set_cell_text(cell_2, "", font_name='Times New Roman', font_size=10, bold=False, alignment='center')

    # Row 2
    cell_1 = table.cell(2, 0)
    cell_2 = table.cell(2, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "SALE PHOTO", font_name='Times New Roman', font_size=10, bold=True, alignment='center')
    set_cell_text(cell_2, "TAX MAP", font_name='Times New Roman', font_size=10, bold=True, alignment='center')

    # Row 3
    cell_1 = table.cell(3, 0)
    cell_2 = table.cell(3, 1)
    cell_1.merge(cell_2)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="D0D0D0")
    set_cell_text(cell_1, "Property Identification", font_name='Times New Roman', font_size=11, bold=True, alignment='center')

    # Row 4
    cell_1 = table.cell(4, 0)
    cell_2 = table.cell(4, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Address", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f'{Address}', font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 5
    cell_1 = table.cell(5, 0)
    cell_2 = table.cell(5, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "City, State", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{City}, {State}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 6
    cell_1 = table.cell(6, 0)
    cell_2 = table.cell(6, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Tax Account Identification", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Tax_ID1}-{Tax_ID2}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 7
    cell_1 = table.cell(7, 0)
    cell_2 = table.cell(7, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Map/Grid/Parcel", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Map_Grid_Parcel}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 8
    cell_1 = table.cell(8, 0)
    cell_2 = table.cell(8, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Legal description", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Legal_Desc}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 9
    cell_1 = table.cell(9, 0)
    cell_2 = table.cell(9, 1)
    cell_1.merge(cell_2)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="D0D0D0")
    set_cell_text(cell_1, "Sale Data", font_name='Times New Roman', font_size=11, bold=True, alignment='center')
    
    # Row 10
    cell_1 = table.cell(10, 0)
    cell_2 = table.cell(10, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Grantor", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Grantor}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 11
    cell_1 = table.cell(11, 0)
    cell_2 = table.cell(11, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Grantee", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Grantee}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 12
    cell_1 = table.cell(12, 0)
    cell_2 = table.cell(12, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Sale Date", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Sale_Date}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 13
    cell_1 = table.cell(13, 0)
    cell_2 = table.cell(13, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Deed Reference", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Deed}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')

    # Row 14
    cell_1 = table.cell(14, 0)
    cell_2 = table.cell(14, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Property Rights", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Property_Rights}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 15
    cell_1 = table.cell(15, 0)
    cell_2 = table.cell(15, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Conditions of Sale", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Conditions_at_Sale}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 16
    cell_1 = table.cell(16, 0)
    cell_2 = table.cell(16, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Time on Market", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Time_on_Market}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 17
    cell_1 = table.cell(17, 0)
    cell_2 = table.cell(17, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Sale Price", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Sale_Price}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 18
    cell_1 = table.cell(18, 0)
    cell_2 = table.cell(18, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Sale Price/GBA", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Price_GBA}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 19
    cell_1 = table.cell(19, 0)
    cell_2 = table.cell(19, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Vacancy", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Vacancy}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 20
    cell_1 = table.cell(20, 0)
    cell_2 = table.cell(20, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Net Operating Income", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{NOI}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 21
    cell_1 = table.cell(21, 0)
    cell_2 = table.cell(21, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Capitalization Rate", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Cap_Rate}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 22
    cell_1 = table.cell(22, 0)
    cell_2 = table.cell(22, 1)
    cell_1.merge(cell_2)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="D0D0D0")
    set_cell_text(cell_1, "Physical Data", font_name='Times New Roman', font_size=11, bold=True, alignment='center')
    
     # Row 23
    cell_1 = table.cell(23, 0)
    cell_2 = table.cell(23, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Land Size (square feet)", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Land_Size_SF}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 24
    cell_1 = table.cell(24, 0)
    cell_2 = table.cell(24, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Gross Building Area", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{GBA_SF}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 25
    cell_1 = table.cell(25, 0)
    cell_2 = table.cell(25, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Year Built", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Year_Built}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')

    # Row 26
    cell_1 = table.cell(26, 0)
    cell_2 = table.cell(26, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Parking", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Parking}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 27
    cell_1 = table.cell(27, 0)
    cell_2 = table.cell(27, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Zoning", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Zone}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 28
    cell_1 = table.cell(28, 0)
    cell_2 = table.cell(28, 1)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_borders(cell_2, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_fill(cell_2, color="FFFFFF")
    set_cell_text(cell_1, "Utilities", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    set_cell_text(cell_2, f"{Utilities}", font_name='Times New Roman', font_size=10, bold=False, alignment='left')
    
    # Row 29
    cell_1 = table.cell(29, 0)
    cell_2 = table.cell(29, 1)
    cell_1.merge(cell_2)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="D0D0D0")
    set_cell_text(cell_1, "Comments", font_name='Times New Roman', font_size=11, bold=True, alignment='center')
    
    # Row 30
    cell_1 = table.cell(30, 0)
    cell_2 = table.cell(30, 1)
    cell_1.merge(cell_2)
    set_cell_borders(cell_1, border_color="000000", border_width=4)
    set_cell_fill(cell_1, color="FFFFFF")
    set_cell_text(cell_1, f"{Comments}", font_name='Times New Roman', font_size=11, bold=False, alignment='left')
    
    # Save the document
    doc.save('extended_table_with_correct_text.docx')

# Main Execution

# Create a new Document
doc = Document()
create_table_with_content(doc)


ValueError: Length mismatch: Expected axis has 34 elements, new values have 33 elements